# Organize freezed data
Aug 19th, 2019

Sep 13th, 2019 (data freeze)

Fangming Xie

- reformat raw data (from the dropbox) into standardized python readable format
- use ensembl id to refer genes
- preserve content as much as possible, including naming conventions of table header and index

In [1]:
import sys
sys.path.insert(0, '/cndd/fangming/CEMBA/snmcseq_dev')
import importlib

from __init__ import *
from __init__jupyterlab import *
import snmcseq_utils
importlib.reload(snmcseq_utils)
import CEMBA_clst_utils
from scipy.io import mmread
from scipy import sparse
import time


In [2]:
SRC_DIR = '/cndd/fangming/CEMBA/data/MOp_all/data_freeze'
DST_DIR = '/cndd/fangming/CEMBA/data/MOp_all/data_freeze_formatted'

In [3]:
f = PATH_GENEBODY_ANNOTATION 
df_genes = pd.read_csv(f, sep="\t")
df_genes['gene_id'] = df_genes['gene_id'].apply(lambda x: x.split('.')[0])
df_genes = df_genes.set_index('gene_id')
df_genes_v2 = df_genes.reset_index().groupby('gene_name').first()
print(df_genes.shape)
df_genes.head()
print(df_genes_v2.shape)
df_genes_v2.head()

(53379, 6)
(53278, 6)


,gene_id,chr,start,end,strand,gene_type
gene_name,,,,,,
0610005C13Rik,ENSMUSG00000109644,chr7,45567794,45575327,-,antisense_RNA
0610006L08Rik,ENSMUSG00000108652,chr7,74818818,74853813,-,lincRNA
0610009B22Rik,ENSMUSG00000007777,chr11,51685386,51688874,-,protein_coding
0610009E02Rik,ENSMUSG00000086714,chr2,26445696,26459390,+,processed_transcript
0610009L18Rik,ENSMUSG00000043644,chr11,120348678,120351190,+,bidirectional_promoter_lncRNA


### mC

- QC'ed cells
- remove version on ensembl id

In [4]:
import xarray as xr

In [6]:
dataset_name = 'snmcseq_gene'

fout_meta = os.path.join(DST_DIR, dataset_name+'_metadata.tsv') 
fout_data_mc = os.path.join(DST_DIR, dataset_name+'_mCG_counts.npz') 
fout_data_c = os.path.join(DST_DIR, dataset_name+'_CG_counts.npz') 
fout_data_gene = os.path.join(DST_DIR, dataset_name+'_counts.gene') 
fout_data_cell = os.path.join(DST_DIR, dataset_name+'_counts.cell') 

print(fout_meta)
print(fout_data_mc)
print(fout_data_c)

/cndd/fangming/CEMBA/data/MOp_all/data_freeze_formatted/snmcseq_gene_metadata.tsv
/cndd/fangming/CEMBA/data/MOp_all/data_freeze_formatted/snmcseq_gene_mCG_counts.npz
/cndd/fangming/CEMBA/data/MOp_all/data_freeze_formatted/snmcseq_gene_CG_counts.npz


In [7]:
# metadata after QC and clustering
# meta = pd.read_msgpack(os.path.join(SRC_DIR, 'snmc_and_snatac/study/mc/Metadata.after_QC.msg'))
# meta2 = pd.read_msgpack(os.path.join(SRC_DIR, 'snmc_and_snatac/study/mc/mop_mc_clustering/MOp.methylome_only.analysis_results.msg')).drop(columns=['cell'])
# print(meta.shape)
# print(meta2.shape)
# print(meta.columns.values.shape)
# print(np.intersect1d(meta.columns.values, meta2.columns.values).shape)
# meta2 = meta2.drop(columns=meta2.filter(regex='^ENSMUSG', axis=1).columns).drop(columns='AllcPath')

meta3 = pd.read_csv(os.path.join(SRC_DIR, 'snmc_and_snatac/study/mc/MOp.snmC-seq.AnalysisResult.csv.gz')).rename(columns={'index': 'cell'})
print(meta3.shape)
meta3.head()

(9876, 32)


,cell,FullCellID,CCC_Rate,CG_Rate,CG_RateAdj,CH_Rate,CH_RateAdj,FinalReads,InputReads,MappedReads,...,Slice,PassFilter,pca_0,pca_1,umap_0,umap_1,tsne_0,tsne_1,MajorCluster,SubCluster
0,2C_M_0,180508_CEMBA_mm_P56_P63_2C_CEMBA180409_2C_1_CE...,0.00511,0.77073,0.76955,0.02154,0.01651,1076259.0,2314862,1517338.0,...,2,True,-11.588627,-0.005045,10.051389,7.099836,-3.343437,18.330174,L4-IT-Rorb,L4-IT-Rorb_Rorb-Tenm2
1,2C_M_1,180508_CEMBA_mm_P56_P63_2C_CEMBA180409_2C_1_CE...,0.00531,0.75683,0.75553,0.02196,0.01674,1197092.0,2590626,1709334.0,...,2,True,-19.267288,-7.374098,13.717241,-1.786627,9.212860,-26.514785,L23-IT-Cux2,L23-IT-Cux2
2,2C_M_10,180508_CEMBA_mm_P56_P63_2C_CEMBA180409_2C_1_CE...,0.00581,0.75720,0.75578,0.02603,0.02034,1006630.0,2348484,1606410.0,...,2,True,-15.340144,-2.739380,4.829485,9.943986,-22.960995,13.509483,Outlier,Outlier
3,2C_M_100,180508_CEMBA_mm_P56_P63_2C_CEMBA180409_2C_1_CE...,0.00634,0.74984,0.74824,0.02718,0.02097,2026078.0,4599922,3154619.0,...,2,True,-23.625140,-5.929033,13.739521,1.107174,12.798545,-17.126331,L23-IT-Cux2,L23-IT-Cux2
4,2C_M_1000,180508_CEMBA_mm_P56_P63_2C_CEMBA180409_2C_3_CE...,0.00521,0.77073,0.76953,0.02117,0.01604,2368464.0,5320502,3648786.0,...,2,True,-21.815190,-7.130878,14.154541,-2.284118,8.443658,-27.943267,L23-IT-Cux2,L23-IT-Cux2


In [8]:
# meta3.to_csv(fout_meta, sep="\t", header=True, index=False)

In [10]:
reps = ['2C-180409.mcds', 
        '2C-180410.mcds',
        '3C-171206.mcds',
        '3C-171207.mcds',
        '4B-171212.mcds',
        '4B-171213.mcds',
        '4B-180104.mcds',
        '5D-180605.mcds',
        '5D-180612.mcds',
       ]

c_tables = []
mc_tables = []
for rep in reps:
    mcds = xr.open_dataset(os.path.join(SRC_DIR, 'snmc_and_snatac/dataset/mc/{}'.format(rep)))
    c_table = mcds['gene_da'].sel(mc_type='CGN', count_type='cov').squeeze().to_pandas()
    mc_table = mcds['gene_da'].sel(mc_type='CGN', count_type='mc').squeeze().to_pandas()
    c_tables.append(c_table)
    mc_tables.append(mc_table)
    
c_tables = pd.concat(c_tables, axis=0)
mc_tables = pd.concat(mc_tables, axis=0)
assert np.all(c_tables.columns.values == mc_tables.columns.values)
assert np.all(c_tables.index.values == mc_tables.index.values)
print(c_tables.shape, mc_tables.shape)
    
gene = c_tables.columns.values
cell = c_tables.index.values 
c_table = c_tables.T
c_table.index = gene

mc_table = mc_tables.T
mc_table.index = gene

(9941, 55487) (9941, 55487)


In [11]:
# remove version on ensembl id
c_table = sparse.coo_matrix(c_table.values)
mc_table = sparse.coo_matrix(mc_table.values)

In [12]:
cell = c_tables.index.values
cell

array(['2C_M_10', '2C_M_0', '2C_M_1', ..., '5D_M_2370', '5D_M_2348',
       '5D_M_2367'], dtype=object)

In [13]:
# save mc and c tables

data = {'mc': mc_table, 
        'c': c_table, 
       }
gc_mat = GC_matrix(gene, cell, data) 

# snmcseq_utils.save_gc_matrix_methylation(gc_mat, 
#                                          fout_data_gene, fout_data_cell, 
#                                          fout_data_mc, fout_data_c)

### mC

- QC'ed cells
- remove version on ensembl id

In [16]:
SRC_DIR = '/cndd/fangming/CEMBA/data/MOp_all/data_freeze_formatted'
DST_DIR = '/cndd/fangming/CEMBA/data/MOp_all/data_freeze_qc_cells'

In [17]:
dataset_name = 'snmcseq_gene'

f_meta = os.path.join(SRC_DIR, dataset_name+'_metadata.tsv') 
f_data_mc = os.path.join(SRC_DIR, dataset_name+'_mCG_raw.npz') 
f_data_c = os.path.join(SRC_DIR, dataset_name+'_CG_raw.npz') 
f_data_gene = os.path.join(SRC_DIR, dataset_name+'_raw.gene') 
f_data_cell = os.path.join(SRC_DIR, dataset_name+'_raw.cell') 

print(f_meta)
print(f_data_mc)
print(f_data_c)
print(f_data_gene)
print(f_data_cell)

fout_meta = os.path.join(DST_DIR, dataset_name+'_metadata.tsv') 
fout_data_mc = os.path.join(DST_DIR, dataset_name+'_mCG_raw.npz') 
fout_data_c = os.path.join(DST_DIR, dataset_name+'_CG_raw.npz') 
fout_data_gene = os.path.join(DST_DIR, dataset_name+'_raw.gene') 
fout_data_cell = os.path.join(DST_DIR, dataset_name+'_raw.cell') 


print(fout_meta)
print(fout_data_mc)
print(fout_data_c)
print(fout_data_gene)
print(fout_data_cell)

/cndd/fangming/CEMBA/data/MOp_all/data_freeze_formatted/snmcseq_gene_metadata.tsv
/cndd/fangming/CEMBA/data/MOp_all/data_freeze_formatted/snmcseq_gene_mCG_raw.npz
/cndd/fangming/CEMBA/data/MOp_all/data_freeze_formatted/snmcseq_gene_CG_raw.npz
/cndd/fangming/CEMBA/data/MOp_all/data_freeze_formatted/snmcseq_gene_raw.gene
/cndd/fangming/CEMBA/data/MOp_all/data_freeze_formatted/snmcseq_gene_raw.cell
/cndd/fangming/CEMBA/data/MOp_all/data_freeze_qc_cells/snmcseq_gene_metadata.tsv
/cndd/fangming/CEMBA/data/MOp_all/data_freeze_qc_cells/snmcseq_gene_mCG_raw.npz
/cndd/fangming/CEMBA/data/MOp_all/data_freeze_qc_cells/snmcseq_gene_CG_raw.npz
/cndd/fangming/CEMBA/data/MOp_all/data_freeze_qc_cells/snmcseq_gene_raw.gene
/cndd/fangming/CEMBA/data/MOp_all/data_freeze_qc_cells/snmcseq_gene_raw.cell


In [18]:
meta = pd.read_csv(f_meta, sep="\t", index_col=0)
# # meta = meta[meta['MajorCluster']!='Outlier']
# print(meta.groupby('MajorCluster').size())
print(meta.shape)
meta.head()

(9876, 31)


,FullCellID,CCC_Rate,CG_Rate,CG_RateAdj,CH_Rate,CH_RateAdj,FinalReads,InputReads,MappedReads,Region,...,Slice,PassFilter,pca_0,pca_1,umap_0,umap_1,tsne_0,tsne_1,MajorCluster,SubCluster
cell,,,,,,,,,,,,,,,,,,,,,
2C_M_0,180508_CEMBA_mm_P56_P63_2C_CEMBA180409_2C_1_CE...,0.00511,0.77073,0.76955,0.02154,0.01651,1076259.0,2314862,1517338.0,2C,...,2,True,-11.588627,-0.005045,10.051389,7.099836,-3.343437,18.330174,L4-IT-Rorb,L4-IT-Rorb_Rorb-Tenm2
2C_M_1,180508_CEMBA_mm_P56_P63_2C_CEMBA180409_2C_1_CE...,0.00531,0.75683,0.75553,0.02196,0.01674,1197092.0,2590626,1709334.0,2C,...,2,True,-19.267288,-7.374098,13.717241,-1.786627,9.212860,-26.514785,L23-IT-Cux2,L23-IT-Cux2
2C_M_10,180508_CEMBA_mm_P56_P63_2C_CEMBA180409_2C_1_CE...,0.00581,0.75720,0.75578,0.02603,0.02034,1006630.0,2348484,1606410.0,2C,...,2,True,-15.340144,-2.739380,4.829485,9.943986,-22.960995,13.509483,Outlier,Outlier
2C_M_100,180508_CEMBA_mm_P56_P63_2C_CEMBA180409_2C_1_CE...,0.00634,0.74984,0.74824,0.02718,0.02097,2026078.0,4599922,3154619.0,2C,...,2,True,-23.625140,-5.929033,13.739521,1.107174,12.798545,-17.126331,L23-IT-Cux2,L23-IT-Cux2
2C_M_1000,180508_CEMBA_mm_P56_P63_2C_CEMBA180409_2C_3_CE...,0.00521,0.77073,0.76953,0.02117,0.01604,2368464.0,5320502,3648786.0,2C,...,2,True,-21.815190,-7.130878,14.154541,-2.284118,8.443658,-27.943267,L23-IT-Cux2,L23-IT-Cux2


In [19]:
# gc_mat = snmcseq_utils.load_gc_matrix_methylation(f_data_gene, f_data_cell, f_data_mc, f_data_c)
print(gc_mat.data['mc'].shape, gc_mat.data['c'].shape, gc_mat.gene.shape, gc_mat.cell.shape)

(55487, 9941) (55487, 9941) (55487,) (9941,)


In [20]:
selected_cells = meta.index.values
selected_cells_abbr = np.array([cell for cell in selected_cells])
selected_cells_id = snmcseq_utils.get_index_from_array(gc_mat.cell, selected_cells_abbr)
selected_cells = dataset_name + '_' + selected_cells
assert np.all(selected_cells_id != -1)

data_update = {'mc': gc_mat.data['mc'].tocsc()[:, selected_cells_id], 
               'c': gc_mat.data['c'].tocsc()[:, selected_cells_id], 
              }

gc_mat_update = GC_matrix(gc_mat.gene, selected_cells, data_update)
print(gc_mat_update.data['mc'].shape, 
      gc_mat_update.data['c'].shape, 
      gc_mat_update.gene.shape, gc_mat_update.cell.shape)

(55487, 9876) (55487, 9876) (55487,) (9876,)


### mC

- QC'ed cells
- remove version on ensembl id

In [22]:
SRC_DIR = '/cndd/fangming/CEMBA/data/MOp_all/data_freeze_qc_cells'
DST_DIR = '/cndd/fangming/CEMBA/data/MOp_all/data_freeze_neurons'

In [29]:
dataset_name = 'snmcseq_gene'

f_meta = os.path.join(SRC_DIR, dataset_name+'_metadata.tsv') 
f_data_mc = os.path.join(SRC_DIR, dataset_name+'_mCG_raw.npz') 
f_data_c = os.path.join(SRC_DIR, dataset_name+'_CG_raw.npz') 
f_data_gene = os.path.join(SRC_DIR, dataset_name+'_raw.gene') 
f_data_cell = os.path.join(SRC_DIR, dataset_name+'_raw.cell') 

print(f_meta)
print(f_data_mc)
print(f_data_c)
print(f_data_gene)
print(f_data_cell)

fout_meta = os.path.join(DST_DIR, dataset_name+'_metadata.tsv') 
fout_data_mc = os.path.join(DST_DIR, dataset_name+'_mCG_raw.npz') 
fout_data_c = os.path.join(DST_DIR, dataset_name+'_CG_raw.npz') 
fout_data_gene = os.path.join(DST_DIR, dataset_name+'_raw.gene') 
fout_data_cell = os.path.join(DST_DIR, dataset_name+'_raw.cell') 


print(fout_meta)
print(fout_data_mc)
print(fout_data_c)
print(fout_data_gene)
print(fout_data_cell)

/cndd/fangming/CEMBA/data/MOp_all/data_freeze_qc_cells/snmcseq_gene_metadata.tsv
/cndd/fangming/CEMBA/data/MOp_all/data_freeze_qc_cells/snmcseq_gene_mCG_raw.npz
/cndd/fangming/CEMBA/data/MOp_all/data_freeze_qc_cells/snmcseq_gene_CG_raw.npz
/cndd/fangming/CEMBA/data/MOp_all/data_freeze_qc_cells/snmcseq_gene_raw.gene
/cndd/fangming/CEMBA/data/MOp_all/data_freeze_qc_cells/snmcseq_gene_raw.cell
/cndd/fangming/CEMBA/data/MOp_all/data_freeze_neurons/snmcseq_gene_metadata.tsv
/cndd/fangming/CEMBA/data/MOp_all/data_freeze_neurons/snmcseq_gene_mCG_raw.npz
/cndd/fangming/CEMBA/data/MOp_all/data_freeze_neurons/snmcseq_gene_CG_raw.npz
/cndd/fangming/CEMBA/data/MOp_all/data_freeze_neurons/snmcseq_gene_raw.gene
/cndd/fangming/CEMBA/data/MOp_all/data_freeze_neurons/snmcseq_gene_raw.cell


In [30]:
meta = pd.read_csv(f_meta, sep="\t", index_col=0)
print(meta.shape)
meta.head()

(9876, 31)


,FullCellID,CCC_Rate,CG_Rate,CG_RateAdj,CH_Rate,CH_RateAdj,FinalReads,InputReads,MappedReads,Region,...,Slice,PassFilter,pca_0,pca_1,umap_0,umap_1,tsne_0,tsne_1,MajorCluster,SubCluster
cell,,,,,,,,,,,,,,,,,,,,,
snmcseq_gene_2C_M_0,180508_CEMBA_mm_P56_P63_2C_CEMBA180409_2C_1_CE...,0.00511,0.77073,0.76955,0.02154,0.01651,1076259.0,2314862,1517338.0,2C,...,2,True,-11.588627,-0.005045,10.051389,7.099836,-3.343437,18.330174,L4-IT-Rorb,L4-IT-Rorb_Rorb-Tenm2
snmcseq_gene_2C_M_1,180508_CEMBA_mm_P56_P63_2C_CEMBA180409_2C_1_CE...,0.00531,0.75683,0.75553,0.02196,0.01674,1197092.0,2590626,1709334.0,2C,...,2,True,-19.267288,-7.374098,13.717241,-1.786627,9.212860,-26.514785,L23-IT-Cux2,L23-IT-Cux2
snmcseq_gene_2C_M_10,180508_CEMBA_mm_P56_P63_2C_CEMBA180409_2C_1_CE...,0.00581,0.75720,0.75578,0.02603,0.02034,1006630.0,2348484,1606410.0,2C,...,2,True,-15.340144,-2.739380,4.829485,9.943986,-22.960995,13.509483,Outlier,Outlier
snmcseq_gene_2C_M_100,180508_CEMBA_mm_P56_P63_2C_CEMBA180409_2C_1_CE...,0.00634,0.74984,0.74824,0.02718,0.02097,2026078.0,4599922,3154619.0,2C,...,2,True,-23.625140,-5.929033,13.739521,1.107174,12.798545,-17.126331,L23-IT-Cux2,L23-IT-Cux2
snmcseq_gene_2C_M_1000,180508_CEMBA_mm_P56_P63_2C_CEMBA180409_2C_3_CE...,0.00521,0.77073,0.76953,0.02117,0.01604,2368464.0,5320502,3648786.0,2C,...,2,True,-21.815190,-7.130878,14.154541,-2.284118,8.443658,-27.943267,L23-IT-Cux2,L23-IT-Cux2


In [31]:
# gc_mat = snmcseq_utils.load_gc_matrix_methylation(f_data_gene, f_data_cell, f_data_mc, f_data_c)
print(gc_mat.data['mc'].shape, gc_mat.data['c'].shape, gc_mat.gene.shape, gc_mat.cell.shape)

(55487, 9941) (55487, 9941) (55487,) (9941,)


In [32]:
# meta[meta]
print(meta.groupby('MajorCluster').size())
print(meta.shape)
meta = meta.loc[meta['MajorCluster'].str.contains(r'^L|^MGE|^CGE')] # removed non-neuronal and outlier
print(meta.shape)
meta['MajorCluster'].unique()


MajorCluster
CGE-VipNdnf      625
L23-IT-Cux2     2057
L4-IT-Rorb      1582
L5-IT-Deptor     898
L5-PT-Bcl6       354
L6-CT-Foxp2     1622
L6-IT-Sulf1      881
L6-NP-Tshz2      275
L6b-Galnt10      141
MGE-Pvalb        460
MGE-Sst          471
NonN             428
Outlier           82
dtype: int64
(9876, 31)
(9366, 31)


array(['L4-IT-Rorb', 'L23-IT-Cux2', 'CGE-VipNdnf', 'L6-IT-Sulf1',
       'L5-IT-Deptor', 'MGE-Sst', 'L6-CT-Foxp2', 'L6-NP-Tshz2',
       'L5-PT-Bcl6', 'MGE-Pvalb', 'L6b-Galnt10'], dtype=object)

In [46]:
selected_cells = meta.index.values
selected_cells_id = snmcseq_utils.get_index_from_array(['snmcseq_gene_' + i for i in gc_mat.cell], 
                                                       selected_cells)
assert np.all(selected_cells_id != -1)

data_update = {'mc': gc_mat.data['mc'].tocsc()[:, selected_cells_id], 
               'c': gc_mat.data['c'].tocsc()[:, selected_cells_id], 
              }

gc_mat_update = GC_matrix(gc_mat.gene, selected_cells, data_update)
print(gc_mat_update.data['mc'].shape, 
      gc_mat_update.data['c'].shape, 
      gc_mat_update.gene.shape, gc_mat_update.cell.shape)

(55487, 9366) (55487, 9366) (55487,) (9366,)


In [47]:
# save mc and c tables

# snmcseq_utils.save_gc_matrix_methylation(gc_mat_update, 
#                                          fout_data_gene, fout_data_cell, 
#                                          fout_data_mc, fout_data_c)

PermissionError: [Errno 13] Permission denied: '/cndd/fangming/CEMBA/data/MOp_all/data_freeze_neurons/snmcseq_gene_raw.gene'

In [56]:
# gc_mat_update.gene[-3:] #.shape
# gc_mat_update.cell.shape
# gc_mat_update.cell[-3:]

# Settings

In [57]:
SRC_DIR = '/cndd/fangming/CEMBA/data/MOp_all/data_freeze_neurons'
DST_DIR = '/cndd/fangming/CEMBA/data/MOp_all/data_freeze_neurons'
sys.path.insert(0, SRC_DIR)
from __init__datasets import *

f_meta_format = '{0}/{1}_metadata.tsv'
f_data_format = '{0}/{1}_{2}raw.{3}'
f_hvftr_format = '{0}/{1}_hvfeatures.{2}'
f_ftr_format = '{0}/{1}_features.{2}'


def get_size_in_GB(obj):
    """"""
    GB = 1024**3
    return sys.getsizeof(obj)/GB

In [58]:
mods_selected = [
    'snmcseq_gene',
#     'snatac_gene',
    
#     'smarter_nuclei',
#     'smarter_cells',
#     '10x_nuclei_v3_macosko',
#     '10x_cells_v3',
#     '10x_nuclei_v3',
#     '10x_cells_v2',
    
#     '10x_nuclei_v2',
#     'smarter-cells-v1',
]

In [59]:
# # gene name as index
# gene_annot_file = PATH_GENEBODY_ANNOTATION
# gene_annot = pd.read_csv(gene_annot_file, sep="\t")
# gene_annot_v2 = gene_annot.groupby('gene_name').first()
# print(gene_annot_v2.shape)
# gene_lengths_base = (gene_annot_v2['end'] - gene_annot_v2['start'])
# print(gene_lengths_base.head())

In [60]:
# # gene id (abbr) as index
gene_annot_file = PATH_GENEBODY_ANNOTATION
gene_annot = pd.read_csv(gene_annot_file, sep="\t")
gene_annot['gene_id_abbr'] = gene_annot['gene_id'].apply(lambda x: x.split('.')[0])
gene_annot = gene_annot.set_index('gene_id_abbr')

gene_lengths_base = (gene_annot['end'] - gene_annot['start'])
print(gene_lengths_base.head())

gene_id_abbr
ENSMUSG00000102693      1069
ENSMUSG00000064842       109
ENSMUSG00000051951    465597
ENSMUSG00000102851       479
ENSMUSG00000103377      2818
dtype: int64


In [61]:
normalization_options = {
    'smarter_nuclei': 'TPM',
    'smarter_cells': 'TPM',
    'snatac_gene': 'TPM',
    '10x_nuclei_v3_macosko': 'CPM',
    '10x_cells_v3': 'CPM',
    '10x_nuclei_v3': 'CPM',
    'snmcseq_gene': 'MC',
    '10x_cells_v2': 'CPM',
#     '10x_nuclei_v2': 'CPM',
#     'smarter-cells-v1': 'TPM',
}

# highly variable features

In [14]:
for mod in mods_selected:
    ti = time.time()
    print(mod)
    
    # read metadata
    normalization_option = normalization_options[mod]
    f_meta = f_meta_format.format(SRC_DIR, mod) ##
    meta = pd.read_csv(f_meta, sep="\t", index_col=0)
    
    # read data matrix
    if normalization_option == 'MC':
        f_data_gene = f_data_format.format(SRC_DIR, mod, '', 'gene')
        f_data_cell = f_data_format.format(SRC_DIR, mod, '', 'cell')
        f_data_c = f_data_format.format(SRC_DIR, mod, 'CG_', 'npz')
        f_data_mc = f_data_format.format(SRC_DIR, mod, 'mCG_', 'npz')
        
        # read in files
        print(mod, "Reading in files {}".format(time.time()-ti))
        gxc_raw = snmcseq_utils.load_gc_matrix_methylation(f_data_gene, f_data_cell, f_data_mc, f_data_c)
        print(gxc_raw.data['mc'].shape, gxc_raw.data['c'].shape)
        print(time.time()-ti)
        
        # output file
        f_hvftr_data_methylation = f_hvftr_format.format(DST_DIR, mod, 'tsv') 
        print(time.time()-ti)
        
        # check meta cells agree with gxc cells
        assert np.all(meta.index.values == gxc_raw.cell)
        # check genes are uniq 
        assert len(gxc_raw.gene) == len(np.unique(gxc_raw.gene)) 
        
        # do
        gxc_hvftr = CEMBA_preproc_utils.preproc_methylation(
                                                            gxc_raw,
                                                            meta,
                                                            global_value_col=settings[mod].global_mean, 
                                                            base_call_cutoff=20, 
                                                            sufficient_coverage_fraction=0.95,
                                                            hv_percentile=30,
                                                            n_qcut=10,
                                                            )
        # save
        print(mod, "Saving to files {}".format(time.time()-ti))
        gxc_hvftr.to_csv(f_hvftr_data_methylation, sep="\t", header=True, index=True, na_rep='NA')
        
        
        
    else:
        f_data = f_data_format.format(SRC_DIR, mod, '', 'npz') 
        f_data_gene = f_data_format.format(SRC_DIR, mod, '', 'gene') 
        f_data_cell = f_data_format.format(SRC_DIR, mod, '', 'cell') 
        
        # read in files
        print(mod, "Reading in files {}".format(time.time()-ti))
        gxc_raw = snmcseq_utils.load_gc_matrix(f_data_gene, f_data_cell, f_data) # checked dimensions in agreement internally
        print(gxc_raw.data.shape)
        
        # output files
        f_hvftr_data = f_hvftr_format.format(DST_DIR, mod, 'npz') 
        f_hvftr_gene = f_hvftr_format.format(DST_DIR, mod, 'gene') 
        f_hvftr_cell = f_hvftr_format.format(DST_DIR, mod, 'cell') 
        
        # check meta cells agree with gxc cells
        assert np.all(meta.index.values == gxc_raw.cell)
        # check genes are uniq 
        assert len(gxc_raw.gene) == len(np.unique(gxc_raw.gene)) 
    
        # get hvftrs
        print(mod, "Preproc and get highly variable genes {}".format(time.time()-ti))
        if normalization_option == 'CPM':
            gxc_hvftr = CEMBA_preproc_utils.preproc_rna_cpm_based(
                                             gxc_raw, 
                                             sufficient_cell_coverage=0.01, 
                                             hv_percentile=30, hv_ncut=10)
            # save
            print(mod, "Saving to files {}".format(time.time()-ti))
            snmcseq_utils.save_gc_matrix(gxc_hvftr, f_hvftr_gene, f_hvftr_cell, f_hvftr_data)

        elif normalization_option == 'TPM':
            gene_lengths = gene_lengths_base.reindex(gxc_raw.gene)
            gxc_hvftr = CEMBA_preproc_utils.preproc_rna_tpm_based(
                                             gxc_raw, gene_lengths, impute_gene_lengths=True, 
                                             sufficient_cell_coverage=0.01, 
                                             hv_percentile=30, hv_ncut=10)
            # save
            print(mod, "Saving to files {}".format(time.time()-ti))
            snmcseq_utils.save_gc_matrix(gxc_hvftr, f_hvftr_gene, f_hvftr_cell, f_hvftr_data)
    
    
    print(mod, "Total time used: {}".format(time.time()-ti))
    

snatac_gene
snatac_gene Reading in files 0.253676176071167
(53342, 54844)
snatac_gene Preproc and get highly variable genes 16.28616428375244
Imputing gene lengths...
Removing low coverage genes...
Getting CPM..
Getting highly variable genes and logCPM...
Getting logTPM...
Trim logTPM matrix...
Number of genes: 7317
snatac_gene Saving to files 87.29555678367615
snatac_gene Total time used: 153.24073886871338


## Check highly-variable genes

In [16]:
for mod in mods_selected:
    print(mod)
    if settings[mod].mod_category == 'mc':
        f_hvftr_data = f_hvftr_format.format(SRC_DIR, mod, 'tsv') 
        gxc_hvftr = pd.read_csv(f_hvftr_data, sep="\t", index_col=0)
        print(gxc_hvftr.index.values)
        print(gxc_hvftr.columns.values)
        print(gxc_hvftr.shape)
        has_nan = np.isnan(gxc_hvftr.values).any()
        print("Contains NaN? {}".format(has_nan))
        
        continue
        
    f_hvftr_data = f_hvftr_format.format(SRC_DIR, mod, 'npz') 
    f_hvftr_gene = f_hvftr_format.format(SRC_DIR, mod, 'gene') 
    f_hvftr_cell = f_hvftr_format.format(SRC_DIR, mod, 'cell') 
    gxc_hvftr = snmcseq_utils.load_gc_matrix(f_hvftr_gene, f_hvftr_cell, f_hvftr_data)
    print(gxc_hvftr.gene)
    print(gxc_hvftr.cell)
    print(len(gxc_hvftr.gene), len(gxc_hvftr.cell), gxc_hvftr.data.shape)
    has_nan = np.isnan(gxc_hvftr.data.data).any()
    print("Contains NaN? {}".format(has_nan))
#     break

snatac_gene
['ENSMUSG00000103147' 'ENSMUSG00000102275' 'ENSMUSG00000099827' ...
 'ENSMUSG00000038344' 'ENSMUSG00000031360' 'ENSMUSG00000031355']
['snatac_gene_CEMBA171206_3C_AGCGATAGAACCAGGTAAGAGATGTATAGCCT'
 'snatac_gene_CEMBA171206_3C_AGCGATAGAACCAGGTAATGACGTCAGGACGT'
 'snatac_gene_CEMBA171206_3C_AGCGATAGAACCAGGTATAGCCTTAGGCGAAG' ...
 'snatac_gene_CEMBA180618_5D_TCCGGAGATTCCATCCGTACTGACAAGAGATG'
 'snatac_gene_CEMBA180618_5D_TCCGGAGATTCCATCCGTACTGACTAAGATCC'
 'snatac_gene_CEMBA180618_5D_TCCGGAGATTCCATCCTATAGCCTCGAATTCC']
7317 54844 (7317, 54844)
Contains NaN? False


### get and save ftrs 

In [70]:
for mod in mods_selected:
    ti = time.time()
    print(mod)
    
    # read metadata
    normalization_option = normalization_options[mod]
    f_meta = f_meta_format.format(SRC_DIR, mod) ##
    meta = pd.read_csv(f_meta, sep="\t", index_col=0)
    
    # read data matrix
    if normalization_option == 'MC':
        f_data_gene = f_data_format.format(SRC_DIR, mod, '', 'gene')
        f_data_cell = f_data_format.format(SRC_DIR, mod, '', 'cell')
        f_data_c = f_data_format.format(SRC_DIR, mod, 'CG_', 'npz')
        f_data_mc = f_data_format.format(SRC_DIR, mod, 'mCG_', 'npz')
        
        # read in files
        print(mod, "Reading in files {}".format(time.time()-ti))
        gxc_raw = snmcseq_utils.load_gc_matrix_methylation(f_data_gene, f_data_cell, f_data_mc, f_data_c)
        print(gxc_raw.data['mc'].shape, gxc_raw.data['c'].shape)
        print(time.time()-ti)
        
        # output file
        f_ftr_data_methylation = f_ftr_format.format(DST_DIR, mod, 'tsv') 
        print(time.time()-ti)
        
        # check meta cells agree with gxc cells
        assert np.all(meta.index.values == gxc_raw.cell)
        # check genes are uniq 
        assert len(gxc_raw.gene) == len(np.unique(gxc_raw.gene)) 
        
        ### do
        base_call_cutoff = 10
        sufficient_coverage_fraction = 0.80
        #  select genes covered (10 counts) in > 80% of cells
        n_gene, n_cell = gxc_raw.data['c'].shape
        gene_cov = (gxc_raw.data['c'] > base_call_cutoff).sum(axis=1)
        gene_cov = np.array(gene_cov).squeeze()/n_cell # fraction of cells covered
        cond = gene_cov>sufficient_coverage_fraction
        print(cond.sum())

        # to full matrix
        df_c = pd.DataFrame(
            gxc_raw.data['c'].tocsr()[cond, :].todense(),
            index=np.array(gxc_raw.gene)[cond],
            columns=gxc_raw.cell,
        )
        df_mc = pd.DataFrame(
            gxc_raw.data['mc'].tocsr()[cond, :].todense(),
            index=np.array(gxc_raw.gene)[cond],
            columns=gxc_raw.cell,
        )
        ### do
        
        # compute normalized methylation matrix 
        df_mcc = snmcseq_utils.get_mcc_lite_v2(df_c, df_mc, base_call_cutoff=base_call_cutoff)
        gxc_ftr = df_mcc.divide(meta.loc[df_mcc.columns.values, settings[mod].global_mean_mcg], axis=1)
        # save
        print(mod, "Saving to files {}".format(time.time()-ti))
        gxc_ftr.to_csv(f_ftr_data_methylation, sep="\t", header=True, index=True, na_rep='NA')
        
        
    else:
        f_data = f_data_format.format(SRC_DIR, mod, '', 'npz') 
        f_data_gene = f_data_format.format(SRC_DIR, mod, '', 'gene') 
        f_data_cell = f_data_format.format(SRC_DIR, mod, '', 'cell') 
        
        # read in files
        print(mod, "Reading in files {}".format(time.time()-ti))
        gxc_raw = snmcseq_utils.load_gc_matrix(f_data_gene, f_data_cell, f_data) # checked dimensions in agreement internally
        print(gxc_raw.data.shape)
        
        # output files
        f_ftr_data = f_ftr_format.format(DST_DIR, mod, 'npz') 
        f_ftr_gene = f_ftr_format.format(DST_DIR, mod, 'gene') 
        f_ftr_cell = f_ftr_format.format(DST_DIR, mod, 'cell') 
        
        # check meta cells agree with gxc cells
        assert np.all(meta.index.values == gxc_raw.cell)
        # check genes are uniq 
        assert len(gxc_raw.gene) == len(np.unique(gxc_raw.gene)) 
    
        # get hvftrs
        print(mod, "Preproc and get highly variable genes {}".format(time.time()-ti))
        if normalization_option == 'CPM':
            # do 
            gxc_ftr = snmcseq_utils.sparse_logcpm(gxc_raw, mode='logcpm')
            # save
            print(mod, "Saving to files {}".format(time.time()-ti))
            snmcseq_utils.save_gc_matrix(gxc_ftr, f_ftr_gene, f_ftr_cell, f_ftr_data)

        elif normalization_option == 'TPM':
            # do
            gene_lengths = gene_lengths_base.reindex(gxc_raw.gene)
            gene_lengths = gene_lengths.fillna(np.nanmean(gene_lengths.values))
            gxc_ftr = snmcseq_utils.sparse_logtpm(gxc_raw, gene_lengths)
            # save
            print(mod, "Saving to files {}".format(time.time()-ti))
            snmcseq_utils.save_gc_matrix(gxc_ftr, f_ftr_gene, f_ftr_cell, f_ftr_data)
    
    
    print(mod, "Total time used: {}".format(time.time()-ti))
    

snmcseq_gene
snmcseq_gene Reading in files 0.11928176879882812
(55487, 9366) (55487, 9366)
107.19905734062195
107.19910740852356
10318
snmcseq_gene Saving to files 265.31904649734497


PermissionError: [Errno 13] Permission denied: '/cndd/fangming/CEMBA/data/MOp_all/data_freeze_neurons/snmcseq_gene_features.tsv'

In [69]:
print(gxc_ftr.shape)
gxc_ftr.head()

(7525, 9366)


,snmcseq_gene_2C_M_0,snmcseq_gene_2C_M_1,snmcseq_gene_2C_M_100,snmcseq_gene_2C_M_1000,snmcseq_gene_2C_M_1001,snmcseq_gene_2C_M_1002,snmcseq_gene_2C_M_1003,snmcseq_gene_2C_M_1004,snmcseq_gene_2C_M_1005,snmcseq_gene_2C_M_1006,...,snmcseq_gene_5D_M_990,snmcseq_gene_5D_M_991,snmcseq_gene_5D_M_992,snmcseq_gene_5D_M_993,snmcseq_gene_5D_M_994,snmcseq_gene_5D_M_995,snmcseq_gene_5D_M_996,snmcseq_gene_5D_M_997,snmcseq_gene_5D_M_998,snmcseq_gene_5D_M_999
ENSMUSG00000051951,0.927889,1.127662,1.041682,1.000709,1.127190,1.056551,0.991620,1.005510,1.124077,1.027323,...,0.946192,1.023840,1.044784,1.059582,1.009258,1.080610,1.063738,1.062934,1.066601,1.057163
ENSMUSG00000102343,1.094741,1.204715,1.274346,1.068506,1.281010,1.110270,1.205094,1.204637,1.094691,1.119629,...,1.127857,1.103604,1.284802,1.169835,1.226061,1.247847,1.131252,1.063990,1.211108,1.066679
ENSMUSG00000025900,1.223751,1.181548,1.276258,1.104059,1.197846,1.136394,1.119016,1.117561,1.157407,1.163425,...,1.138709,1.133095,1.103612,1.162994,1.123889,1.177215,1.081222,0.960251,1.110183,1.156682
ENSMUSG00000025903,1.245572,1.165854,1.244710,0.778483,0.945507,1.202052,1.173793,1.113108,1.005291,1.046020,...,1.009966,1.051051,0.817601,0.998947,1.037436,1.247847,1.165487,1.024973,1.211108,1.132176
ENSMUSG00000104217,1.254222,0.643711,1.111348,0.835659,0.951902,0.858609,1.110408,0.878629,0.983584,1.018974,...,1.182739,1.128128,1.107588,1.231406,0.886938,1.195854,1.131667,1.218363,1.211108,1.163501


In [71]:
f = '/cndd/fangming/CEMBA/data/MOp_all/data_freeze_neurons/snmcseq_gene_features_mcg.tsv'
gxc_ftr.to_csv(f, sep="\t", header=True, index=True, na_rep='NA')